# Information Retrieval project
**Authors:** Arduini L., Menchini L., Namaki Ghaneh D., Petruzzella C.

# 1. Preprocessing

In [4]:
!pip install ir_datasets
!pip install nltk
!pip install ir_measures

In [5]:
import ir_datasets

# Load the MS MARCO dataset
dataset = ir_datasets.load("msmarco-passage")
# dataset = ir_datasets.load("vaswani")

In [6]:
# print the first document in the dataset

import random

# Initialize a flag to check if a document has been printed
document_printed = False

# Iterate over the documents in the dataset
for doc in dataset.docs_iter():
    if not document_printed:
        if random.random() < 0.01:  # Adjust the probability as needed
            print(doc.text)
            document_printed = True
            break

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 3417: character maps to <undefined>

In [4]:
import re
import string
import nltk

nltk.download("stopwords", quiet=True)

# Compile regex patterns once globally
ACRONYM_REGEX = re.compile(r"(?<!\w)\.(?!\d)")
PUNCTUATION_TRANS = str.maketrans("", "", string.punctuation)

# Preload stopwords set
STOPWORDS = set(nltk.corpus.stopwords.words('english'))

# Initialize stemmer
STEMMER = nltk.stem.PorterStemmer()

def preprocess(s):
    # lowercasing
    s = s.lower()

    # replace ampersand
    s = s.replace("&", " and ")

    # normalize quotes and dashes
    s = s.translate(str.maketrans("‘’´“”–-", "'''\"\"--"))

    # remove unnecessary dots in acronyms (but not decimals)
    s = ACRONYM_REGEX.sub("", s)

    # remove punctuation
    s = s.translate(PUNCTUATION_TRANS)

    # strip and remove extra spaces
    s = " ".join(s.split())

    # tokenize
    tokens = s.split()

    # remove stopwords
    tokens = [t for t in tokens if t not in STOPWORDS]

    # stemming
    tokens = [STEMMER.stem(t) for t in tokens]

    return tokens


In [5]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [6]:
from collections import Counter
from tqdm.auto import tqdm

@profile
def build_index(dataset):
    lexicon = {}
    doc_index = []
    inv_d, inv_f = {}, {}
    termid = 0

    num_docs = 0
    total_dl = 0
    total_toks = 0
    for docid, doc in tqdm(enumerate(dataset.docs_iter()), desc='Indexing', total=dataset.docs_count()):
        tokens = preprocess(doc.text)
        token_tf = Counter(tokens)
        for token, tf in token_tf.items():
            if token not in lexicon:
                lexicon[token] = [termid, 0, 0]
                inv_d[termid], inv_f[termid] =  [], []
                termid += 1
            token_id = lexicon[token][0]
            inv_d[token_id].append(docid)
            inv_f[token_id].append(tf)
            lexicon[token][1] += 1
            lexicon[token][2] += tf
        doclen = len(tokens)
        doc_index.append((str(doc.doc_id), doclen))
        total_dl += doclen
        num_docs += 1


    stats = {
        'num_docs': 1 + docid,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

c:\Users\loren\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Compress and save the index components

In [7]:
#
# check if files exists in the current directory, if not we build the index and we save the files
#

import gzip
import pickle
import os

lex, inv, doc, stats = None, None, None, None

files_to_check = ['lexicon.pickle.gz', 'inverted_file.pickle.gz', 'document_index.pickle.gz', 'stats.pickle.gz']
if all(os.path.exists(file) for file in files_to_check):
    print("All files already exist.")
    
    for file, var_name in zip(files_to_check, ['lex', 'inv', 'doc', 'stats']):
        try:
            if os.path.getsize(file) > 0:  # Verifica se il file non è vuoto
                with gzip.open(file, 'rb') as f:
                    globals()[var_name] = pickle.load(f)
            else:
                print(f"Warning: {file} is empty.")
        except EOFError:
            print(f"Error: {file} is corrupted or incomplete. Rebuilding the index.")
            lex, inv, doc, stats = build_index(dataset)
            break
else:
    # Se i file non esistono o sono corrotti, ricostruisci l'indice
    lex, inv, doc, stats = build_index(dataset)

    # Salva nuovamente i dati nei file compressi solo se necessario
    print("saving files")
    for data, file in zip([lex, inv, doc, stats], files_to_check):
      with gzip.open(file, 'wb') as f:
        pickle.dump(data, f)


Indexing: 100%|██████████| 11429/11429 [00:06<00:00, 1735.69it/s]


build_index (6586.105 ms)


In [8]:
import math

class InvertedIndex:

    class PostingListIterator:
        def __init__(self, docids, freqs, doc):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc

        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]

        def score(self):
            if self.is_end_list():
                return math.inf
            return self.freqs[self.pos]/self.doc[self.docid()][1]

        def next(self, target = None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    try:
                        self.pos = self.docids.index(target, self.pos)
                    except ValueError:
                        self.pos = len(self.docids)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)


    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stat = stats

    def num_docs(self):
        return self.stats['num_docs']

    def get_posting(self, termid):
        return InvertedIndex.PostingListIterator(self.inv['docids'][termid], self.inv['freqs'][termid], self.doc)

    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]

In [9]:
inv_index = InvertedIndex(lex, inv, doc, stats)

# 3. Query processing

Decompress and load the index components


In [10]:
import gzip
import pickle

with gzip.open('lexicon.pickle.gz', 'rb') as f:
    lex = pickle.load(f)
with gzip.open('inverted_file.pickle.gz', 'rb') as f:
    inv = pickle.load(f)
with gzip.open('document_index.pickle.gz', 'rb') as f:
    doc = pickle.load(f)
with gzip.open('stats.pickle.gz', 'rb') as f:
    stats = pickle.load(f)

In [11]:
trec_dl_2020 = ir_datasets.load("msmarco-passage/trec-dl-2020")
#trec_dl_2020 = ir_datasets.load("vaswani")

### Ranked Retrieval

In [12]:
import heapq

class TopQueue:
    def __init__(self, k=10, threshold=0.0):
        self.queue = []
        self.k = k
        self.threshold = threshold

    def size(self):
        return len(self.queue)

    def would_enter(self, score):
        return score > self.threshold

    def clear(self, new_threshold=None):
        self.queue = []
        if new_threshold:
            self.threshold = new_threshold

    def __repr__(self):
        return f'<{self.size()} items, th={self.threshold} {self.queue}'

    def insert(self, docid, score):
        if score > self.threshold:
            if self.size() >= self.k:
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True
        return False

BM25 scoring function

In [13]:
avg_dl = inv_index.stat['num_tokens'] / inv_index.stat['num_docs']
N = inv_index.stat['num_docs']

# BM25 for a term
def bm25(tf, df, dl, k1=1.5, b=0.75):
    idf = math.log(1 + (N - df + 0.5) / (df + 0.5))
    term_frequency_component = (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (dl / avg_dl)))
    return idf * term_frequency_component

DAAT with BM25

In [14]:
import math
from collections import defaultdict

# Calculate document lengths
doc_lengths = defaultdict(int)
for docid, doc_len in inv_index.doc:
    doc_lengths[docid] = doc_len

def min_docid(postings):
    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat_bm25(postings, k=10):
    top = TopQueue(k)
    current_docid = min_docid(postings)

    while current_docid != math.inf:
        score = 0
        next_docid = math.inf

        for posting in postings:
            if posting.docid() == current_docid:
                tf = posting.freqs[posting.pos]
                df = posting.len()
                dl = doc_lengths[current_docid]

                score += bm25(tf, df, dl)

                posting.next()
            if not posting.is_end_list():
                next_docid = min(next_docid, posting.docid())

        top.insert(current_docid, score)
        current_docid = next_docid

    return sorted(top.queue, reverse=True)

TAAT with BM25

In [15]:
def taat_bm25(postings, k=10):
    A = defaultdict(float)

    for posting in postings:
        current_docid = posting.docid()

        df = posting.len()

        while current_docid != math.inf:
            tf = posting.freqs[posting.pos]
            dl = doc_lengths[current_docid]

            score = bm25(tf, df, dl)
            A[current_docid] += score

            posting.next()
            current_docid = posting.docid()

    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)

    return sorted(top.queue, reverse=True)

In [16]:
@profile
def query_processing(queries_iter, fn):
    res = []
    for q in queries_iter:
        query = preprocess(q.text)
        termids = inv_index.get_termids(query)
        postings = inv_index.get_postings(termids)
        res.append({'query_id': q.query_id, 'scores': fn(postings)})
    return res

In [17]:
print(query_processing(trec_dl_2020.queries_iter(), daat_bm25))

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2020-queries.tsv.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2020-queries.tsv.gz: [00:00] [4.13kB] [5.13MB/s]


query_processing (2357.330 ms)
[{'query_id': '1030303', 'scores': []}, {'query_id': '1037496', 'scores': []}, {'query_id': '1043135', 'scores': [(16.251875724873987, 2145), (14.711334160533614, 6782), (14.711334160533614, 6492), (14.711334160533614, 16)]}, {'query_id': '1045109', 'scores': [(16.251875724873987, 10498)]}, {'query_id': '1049519', 'scores': [(15.379284126873308, 5177), (14.711334160533614, 10774), (14.711334160533614, 8364), (14.711334160533614, 8325), (14.711334160533614, 6794), (14.711334160533614, 5146), (14.711334160533614, 2098), (14.49298150036191, 7868), (14.49298150036191, 7279), (14.49298150036191, 6336)]}, {'query_id': '1051399', 'scores': []}, {'query_id': '1056416', 'scores': [(18.49081942401254, 10890), (18.49081942401254, 4989), (18.49081942401254, 4459), (11.684047317644685, 7962), (11.684047317644685, 6846), (11.684047317644685, 4992), (11.684047317644685, 4974), (11.684047317644685, 4927), (11.684047317644685, 4046), (11.684047317644685, 1942)]}, {'query_

In [18]:
results = query_processing(trec_dl_2020.queries_iter(), taat_bm25)
print(results)

query_processing (143.314 ms)
[{'query_id': '1030303', 'scores': []}, {'query_id': '1037496', 'scores': []}, {'query_id': '1043135', 'scores': [(16.251875724873987, 2145), (14.711334160533614, 6782), (14.711334160533614, 6492), (14.711334160533614, 16)]}, {'query_id': '1045109', 'scores': [(16.251875724873987, 10498)]}, {'query_id': '1049519', 'scores': [(15.379284126873308, 5177), (14.711334160533614, 10774), (14.711334160533614, 8364), (14.711334160533614, 8325), (14.711334160533614, 6794), (14.711334160533614, 5146), (14.711334160533614, 2098), (14.49298150036191, 8386), (14.49298150036191, 7868), (14.49298150036191, 7279)]}, {'query_id': '1051399', 'scores': []}, {'query_id': '1056416', 'scores': [(18.49081942401254, 10890), (18.49081942401254, 4989), (18.49081942401254, 4459), (11.684047317644685, 7962), (11.684047317644685, 6846), (11.684047317644685, 4992), (11.684047317644685, 4974), (11.684047317644685, 4927), (11.684047317644685, 4046), (11.684047317644685, 1942)]}, {'query_i

# 4. Evaluation

In [19]:
for doc in dataset.docs_iter()[:3]:
    print(doc)

GenericDoc(doc_id='1', text='compact memories have flexible capacities  a digital data storage\nsystem with capacity up to bits and random and or sequential access\nis described\n')
GenericDoc(doc_id='2', text='an electronic analogue computer for solving systems of linear equations\nmathematical derivation of the operating principle and stability\nconditions for a computer consisting of amplifiers\n')
GenericDoc(doc_id='3', text='electronic coordinate transformer  circuit details are given for\nthe construction of an electronic calculating unit which enables\nthe polar coordinates of a vector modulus and cosine or sine of the\nargument to be derived from those of a rectangular system of axes\n')


In [20]:
trec_dl_2020 = ir_datasets.load("msmarco-passage/trec-dl-2020")
for query in trec_dl_2020.queries_iter()[:3]:
    print(query) # namedtuple<query_id, text>

GenericQuery(query_id='1030303', text='who is aziz hashim')
GenericQuery(query_id='1037496', text='who is rep scalise?')
GenericQuery(query_id='1043135', text='who killed nicholas ii of russia')


In [21]:
for ass in list(trec_dl_2020.qrels_iter())[:3]:
  print(ass)

[INFO] [starting] https://trec.nist.gov/data/deep/2020qrels-pass.txt
[INFO] [finished] https://trec.nist.gov/data/deep/2020qrels-pass.txt: [00:00] [219kB] [228kB/s]
                                                                             

TrecQrel(query_id='23849', doc_id='1020327', relevance=2, iteration='0')
TrecQrel(query_id='23849', doc_id='1034183', relevance=3, iteration='0')
TrecQrel(query_id='23849', doc_id='1120730', relevance=0, iteration='0')


In [22]:
# Generate run file
trec_run_list = []
for doc_scores in results:
    rank = 1
    query_id = doc_scores['query_id']
    scores = doc_scores['scores']

    for score, doc_id in scores:
        line = f"{query_id} Q0 {doc_id} {rank} {score} GOODFELLAS"
        trec_run_list.append(line)
        rank += 1

with open("trec_eval_run_file.txt", "w") as f:
    for line in trec_run_list:
        f.write(line + "\n")

In [23]:
# Create format for Trec_Eval
qrels_file = []
for qrel in trec_dl_2020.qrels_iter():
    line = f"{qrel.query_id} 0 {qrel.doc_id} {qrel.relevance}"
    qrels_file.append(line)

In [24]:
with open("trec_eval_qrels_file.txt", "w") as f:
    for line in qrels_file:
        f.write(line + "\n")

In [25]:
import ir_measures
qrels = ir_measures.read_trec_qrels('trec_eval_qrels_file.txt')
run = ir_measures.read_trec_run('trec_eval_run_file.txt')

In [26]:
measures = ir_measures.P@10, ir_measures.R@1000, ir_measures.AP, ir_measures.nDCG@10
results = ir_measures.calc_aggregate(measures, qrels, run)
print(results)

{nDCG@10: 0.0, R@1000: 0.0, AP: 0.0, P@10: 0.0}
